In [1]:
from keras.layers.core import Dense, Dropout, SpatialDropout1D
from keras.layers.convolutional import Conv1D
from keras.layers.embeddings import Embedding
from keras.layers.pooling import GlobalMaxPooling1D
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
import collections
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd

np.random.seed(42)

/Users/taiwoalabi/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
INPUT_FILE = "/Users/taiwoalabi/Downloads/winemag-data-130k-v2.csv"
TOKEN_GO = '<GO>'
TOKEN_EOS = '<EOS>'
TOKEN_PAD = '<PAD>'
TOKEN_UNK = '<UNK>'

In [3]:
Marie= pd.read_csv(INPUT_FILE)

In [4]:
for ii in range(len(Marie)):
    Marie.loc[ii,"Points1"] = int(Marie.loc[ii,"points"])/100

In [5]:
for ii in range(len(Marie)):
    if Marie.loc[ii,"Points1"] > 0.95:
        Marie.loc[ii, "Rating_Val"]= "AA"
    
    elif (Marie.loc[ii, "Points1"] > 0.9) & (Marie.loc[ii, "Points1"] <= 0.95):
        Marie.loc[ii, "Rating_Val"]= "BB"
        
    
    elif (Marie.loc[ii, "Points1"] > 0.85) & (Marie.loc[ii, "Points1"] <= 0.9):
        Marie.loc[ii, "Rating_Val"]= "CC"
        
    elif (Marie.loc[ii, "Points1"] <= 0.85):
        Marie.loc[ii, "Rating_Val"]= "DD"

In [6]:
Adjective_Nouns= ['JJ', 'JJR','JJS','NN', 'NNS', 'NNP', 'NNPS', 'VBN' ]
for ii in range(len(Marie)):
    Z= nltk.pos_tag(nltk.word_tokenize(Marie.loc[ii, "description"]))
    ZZ= [ii[0] for ii in Z if ii[1] in Adjective_Nouns]
    Marie.loc[ii,"POS-Tag"]= ' '.join(ZZ)

### Sequence translation from this value ###

In [7]:
for ii in range(len(Marie)):
    Marie.loc[ii,"Machine_Trans1"] = str("<GO>") + " " + str(Marie.loc[ii, "Rating_Val"]) + " " + str(Marie.loc[ii,"country"]) + " " + str(Marie.loc[ii,"province"]) \
    + " " + str(Marie.loc[ii,"variety"]) + "."

### Sequence translating to this value ###

In [8]:
for ii in range(len(Marie)):
    Marie.loc[ii, "description1"] = str(Marie.loc[ii, "description"]) + " " + "<EOS>"

#### Creating the Word Indice(s) and the Embedding Matrix -- Input Vocabulary

In [9]:
counter = collections.Counter()
maxlen = 0
for iii in range(len(Marie)):
    sentence = Marie['Machine_Trans1'][iii]
    words = [x.lower() for x in nltk.word_tokenize(sentence)]
    
    if len(words) > maxlen:
        maxlen = len(words)
    for word in words:
        counter[word] += 1

In [31]:
Z= word2index["<GO>"]

In [32]:
Z

0

In [11]:
print("This is the maximum length of the Input Vocabulary sentences:", maxlen)

This is the maximum length of the Input Vocabulary sentences: 13


In [10]:
VOCAB_SIZE = 10000
word2index = collections.defaultdict(int)
for wid, word in enumerate(counter.most_common()):
    word2index[word[0]] = wid + 1
vocab_size = len(word2index) + 1
index2word = {v:k for k, v in word2index.items()}

In [11]:
print("This is the vocabulary size of the entire Input-Vocabulary: ", vocab_size)

This is the vocabulary size of the entire Input-Vocabulary:  1166


In [ ]:
#### - VOCAB_SIZE must be - 1200
# vocab_size = 1200

#### Creating the Word Indice(s) and the Embedding Matrix -- Target Vocabulary

In [12]:
counter = collections.Counter()
maxlen = 0
for iii in range(len(Marie)):
    sentence = Marie['description1'][iii]
    words = [x.lower() for x in nltk.word_tokenize(sentence)]
    
    if len(words) > maxlen:
        maxlen = len(words)
    for word in words:
        counter[word] += 1

In [14]:
print("This is the maximum length of the target Vocabulary sentences: ", maxlen)

This is the maximum length of the target Vocabulary sentences:  159


In [13]:
VOCAB_SIZE2 = 5000
word2index2 = collections.defaultdict(int)
for wid, word in enumerate(counter.most_common()):
    word2index2[word[0]] = wid + 1
vocab_size2 = len(word2index2) + 1
index2word2 = {v:k for k, v in word2index2.items()}

In [16]:
print("This is the vocabulary size of the entire target-Vocabulary: ", vocab_size2)

This is the vocabulary size of the entire target-Vocabulary:  45967


In [17]:
#### - VOCAB_SIZE must be - 46000

In [14]:
xs,ys = [], []

for iii in range(len(Marie)):
    sentence = Marie['Machine_Trans1'][iii]
    sentence2 = Marie['description1'][iii]
    words = [x.lower() for x in nltk.word_tokenize(sentence)]
    wids = [word2index[word] for word in words]
    
    words2 = [x.lower() for x in nltk.word_tokenize(sentence2)]
    wids2 = [word2index2[word] for word in words2]
    xs.append(wids)
    ys.append(wids2)

X = pad_sequences(xs, maxlen=160) #X is the input i.e the wine rating, country, variety etc.
Y = pad_sequences(ys, maxlen=160) #Y is the ouput i.e the description of the wine etc.




In [36]:
Marie['Machine_Trans1'][0]

'<GO> CC Italy Sicily & Sardinia White Blend.'

In [37]:
X_test[0]

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  2,  3,  5,
       10, 61, 60, 62, 29,  9,  4], dtype=int32)

In [38]:
Marie['description1'][0]

"Aromas include tropical fruit, broom, brimstone and dried herb. The palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity. <EOS>"

In [40]:
Y_test[0]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
          22,   854,

In [15]:
Xtrain_id, Xtest_id, Ytrain_id, Ytest_id = train_test_split(X, Y, test_size=0.3, random_state=50)

In [16]:
XtrainReal = Xtrain_id[1:6000]
YtrainReal = Ytrain_id[1:6000]

XtestReal = Xtest_id[6000:10000]
YtestReal = Ytest_id[6000:10000]

In [17]:
X_true = X[1:20000]
Y_true = Y[1:20000]

In [18]:
X_test= X[0:101]
Y_test=Y[0:101]

In [22]:
len(X_true)

59999

## Neural Machine Translation without the Attention mechanism ##

In [19]:
import tensorflow as tf
import json, os, re, shutil, sys, time
import numpy as np

In [20]:
tf.reset_default_graph()
tf.set_random_seed(42)

Src_Vocab_Size = 1200
embedding_size = 500
num_units = 500
learning_rate = 0.001

trg_Vocab_Size = 46000
embedding_size2 = 500

#Max encoder time vs batch size
encoder_inputs = tf.placeholder(tf.int32, [None, None], name = "Encoder_input")

#Max decoder time vs batch size
decoder_inputs = tf.placeholder(tf.int32, [None, None], name = "Decoder_input")

#Decoder output - These are the decoder output shifted to the left by one time step with an end-of-sentence tag appended to the right - max_decoder_time vs batch_size
decoder_outputs = tf.placeholder(tf.int32, [None, None], name = "Decoder_output")

In [21]:
#Getting the batch-size
with tf.name_scope("Encode_batch_size"):
    batch_size = tf.shape(encoder_inputs)[1]
    
with tf.name_scope("Decoder_max_time"):
    max_time_decode= tf.shape(decoder_inputs)[0]
    
with tf.name_scope("Decode_batch_size"):
    batch_size_decode = tf.shape(decoder_inputs)[1]

#Getting the max-time
with tf.name_scope("Encode_max_time"):
    max_time = tf.shape(encoder_inputs)[0]
    
    ns_ = tf.tile([max_time],[batch_size,], name = "ns")
    ns_2 = tf.tile([max_time_decode],[batch_size_decode,], name = "ns2")

with tf.name_scope("Embedding_Layer_Encoder"):
    embedding_encoder = tf.Variable(tf.random_uniform([Src_Vocab_Size,embedding_size], -1.0, 1.0), name="Embed_Encoder")
    encoder_emb_inp = tf.nn.embedding_lookup(embedding_encoder,encoder_inputs)

with tf.name_scope("Embedding_Layer_Decoder"):
    embedding_decoder = tf.Variable(tf.random_uniform([trg_Vocab_Size,embedding_size2], -1.0, 1.0), name="Embed_decoder")
    decoder_emb_inp = tf.nn.embedding_lookup(embedding_decoder,decoder_inputs)
    

with tf.name_scope("Encoder_RNN_Cell"):

    
    encoder_cell = tf.nn.rnn_cell.BasicLSTMCell(num_units = num_units)
    #initial_h_ = encoder_cell.zero_state(batch_size, dtype = tf.float32)
    encoder_outputs, encoder_state = tf.nn.dynamic_rnn(encoder_cell, encoder_emb_inp, sequence_length = ns_, time_major = True, dtype= tf.float32)
    
    
with tf.name_scope("Decode_LSTM_Cell"):
    projection_layer = tf.layers.Dense(trg_Vocab_Size, use_bias = False)
    decoder_cell = tf.nn.rnn_cell.BasicLSTMCell(num_units)
    helper = tf.contrib.seq2seq.TrainingHelper(decoder_emb_inp, ns_2, time_major = True)
    decoder = tf.contrib.seq2seq.BasicDecoder(decoder_cell, helper, encoder_state, output_layer = projection_layer)

with tf.name_scope("Dynamic_Decoding"):
    outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(decoder)
    logits = outputs.rnn_output
    
with tf.name_scope("Cost_Function"):
    crossent = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = decoder_outputs, logits = logits)
    train_loss = tf.reduce_mean(crossent, name = "sparse_softmax")
    
with tf.name_scope("Gradient_Optimization"):
    params = tf.trainable_variables()
    gradients = tf.gradients(train_loss, params)
    clipped_gradients, _ = tf.clip_by_global_norm(gradients, 1)
    
    alpha_ = tf.placeholder(tf.float32, name="learning_rate")
    optimizer = tf.train.AdamOptimizer(alpha_)
    update_step = optimizer.apply_gradients(zip(clipped_gradients,params))
    
    

Instructions for updating:
Use the retry module or similar alternatives.


In [22]:
# Initializer step
init_ = tf.global_variables_initializer()

In [23]:
summary_writer = tf.summary.FileWriter("~/Desktop/Machine_Learning_Projects", 
                                       tf.get_default_graph())

In [24]:
def multi_batch_generator(batch_size, *data_arrays):
    """Generate minibatches from multiple columns of data.

    Example:
        for (bx, by) in multi_batch_generator(5, x, y):
            # bx is minibatch for x
            # by is minibatch for y

    Args:
      batch_size: int, batch size
      data_arrays: one or more array-like, supporting slicing along the first
        dimension, and with matching first dimension.

    Yields:
      minibatches of maximum size batch_size
    """
    assert(data_arrays)
    num_examples = len(data_arrays[0])
    for i in range(1, len(data_arrays)):
        assert(len(data_arrays[i]) == num_examples)

    for i in range(0, num_examples, batch_size):
        # Yield matching slices from each data array.
        yield tuple(data[i:i+batch_size] for data in data_arrays)

In [25]:
import numpy as np
def run_epoch(session, batch_iterator,
              train=False, verbose=False,
              tick_s=10, learning_rate=None):
    assert(learning_rate is not None)
    start_time = time.time()
    tick_time = start_time  # for showing status
    total_cost = 0.0  # total cost, summed over all words
    total_batches = 0
    total_words = 0

    if train:
        train_op = update_step
        
        loss = train_loss
    else:
        train_op = tf.no_op()
        use_dropout = False  # no dropout at test time
        loss = train_loss  # true loss, if train_loss is an approximation

    for i, (x, y) in enumerate(batch_iterator):
        cost = 0.0
        XX= np.transpose(y)
        #print(XX)
        #print(x)
        #print(y)
        
        feed_dict = {encoder_inputs:x, decoder_inputs:y,alpha_:learning_rate,decoder_outputs:XX}
        _, cost = session.run([train_op, loss],
                       feed_dict=feed_dict)
        

        #### END(YOUR CODE) ####
        total_cost += cost
        total_batches = i + 1
        total_words += x.size  # w.size = batch_size * max_time

        ##
        # Print average loss-so-far for epoch
        # If using train_loss_, this may be an underestimate.
        if verbose and (time.time() - tick_time >= tick_s):
            avg_cost = total_cost / total_batches
            avg_wps = total_words / (time.time() - start_time)
            print("[batch {:d}]: seen {:d} words at {:.1f} wps, loss = {:.3f}".format(
                i, total_words, avg_wps, avg_cost))
            tick_time = time.time()  # reset time ticker

    return total_cost / total_batches

In [26]:
def score_dataset(session, X, Y, name="Data"):
    # For scoring, we can use larger batches to speed things up.
    batch_size =200
    bi = multi_batch_generator(batch_size, X, Y)
    cost = run_epoch(session, bi, 
                     learning_rate=0.01, train=False, 
                     verbose=False, tick_s=3600)
    print("{:s}: avg. loss: {:.03f}  (perplexity: {:.02f})".format(name, cost, np.exp(cost)))
    return cost

In [27]:
def pretty_timedelta(fmt="%d:%02d:%02d", since=None, until=None):
    """Pretty-print a timedelta, using the given format string."""
    since = since or time.time()
    until = until or time.time()
    delta_s = until - since
    hours, remainder = divmod(delta_s, 3600)
    minutes, seconds = divmod(remainder, 60)
    return fmt % (hours, minutes, seconds)

In [28]:
TF_SAVEDIR = "Desktop/Machine_Learning_Projects/"
checkpoint_filename = os.path.join(TF_SAVEDIR, "NMT")
trained_filename = os.path.join(TF_SAVEDIR, "NMT_trained")

In [29]:
num_epochs = 10
batch_size = 200
learning_rate = 0.01  
print_every = 1000
saver = tf.train.Saver()
np.random.seed(42)

session = tf.Session()
session.run(init_)

t0 = time.time()
for epoch in range(1,num_epochs+1):
    t0_epoch = time.time()
    epoch_cost = 0.0
    total_batches = 0
    print ("")
    for epoch in range(1,num_epochs+1):
        t0_epoch = time.time()
        bi = multi_batch_generator(batch_size, X_true, Y_true)
        print("[epoch {:d}] Starting epoch {:d}".format(epoch, epoch))
        #### YOUR CODE HERE ####
        #(lm, session, batch_iterator,
              #train=False, verbose=False,
              #tick_s=10, learning_rate=None)
        # Run a training epoch.
        run_epoch(session, bi, train=True, tick_s=10,learning_rate =learning_rate, verbose= True)
        
        #### END(YOUR CODE) ####
        print("[epoch {:d}] Completed in {:s}".format(epoch, pretty_timedelta(since=t0_epoch)))
    
        # Save a checkpoint
        saver.save(session, checkpoint_filename, global_step=epoch)
    
        ##
        # score_dataset will run a forward pass over the entire dataset
        # and report perplexity scores. This can be slow (around 1/2 to 
        # 1/4 as long as a full epoch), so you may want to comment it out
        # to speed up training on a slow machine. Be sure to run it at the 
        # end to evaluate your score.
        print("[epoch {:d}]".format(epoch), end=" ")
        score_dataset(session, XtrainReal, YtrainReal, name="Train set")
        print("[epoch {:d}]".format(epoch), end=" ")
        score_dataset(session, XtestReal, XtestReal, name="Test set")
        print("")
    
    # Save final model
    saver.save(session, trained_filename)


[epoch 1] Starting epoch 1
[batch 0]: seen 32000 words at 96.8 wps, loss = 10.727
[batch 1]: seen 64000 words at 100.0 wps, loss = 8.754
[batch 2]: seen 96000 words at 100.8 wps, loss = 7.096
[batch 3]: seen 128000 words at 102.2 wps, loss = 5.998
[batch 4]: seen 160000 words at 103.2 wps, loss = 5.307
[batch 5]: seen 192000 words at 103.9 wps, loss = 4.869
[batch 6]: seen 224000 words at 104.0 wps, loss = 4.531
[batch 7]: seen 256000 words at 104.7 wps, loss = 4.293
[batch 8]: seen 288000 words at 104.5 wps, loss = 4.069
[batch 9]: seen 320000 words at 104.4 wps, loss = 3.879
[batch 10]: seen 352000 words at 105.4 wps, loss = 3.709
[batch 11]: seen 384000 words at 106.4 wps, loss = 3.564
[batch 12]: seen 416000 words at 107.3 wps, loss = 3.449
[batch 13]: seen 448000 words at 107.5 wps, loss = 3.328
[batch 14]: seen 480000 words at 107.2 wps, loss = 3.229
[batch 15]: seen 512000 words at 107.2 wps, loss = 3.146
[batch 16]: seen 544000 words at 108.1 wps, loss = 3.067
[batch 17]: seen

KeyboardInterrupt: 

### This is the inference step for decoding ###

In [54]:


helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(embedding_decoder, tf.fill([batch_size],9),894)

decoder = tf.contrib.seq2seq.BasicDecoder(decoder_cell, helper, encoder_state, output_layer=projection_layer)
maximum_iterations =tf.round(tf.reduce_max(ns_)*2)

outputs, _, _= tf.contrib.seq2seq.dynamic_decode(decoder)
translations =outputs.sample_id

